In [ ]:
from itertools import chain
from deepface import DeepFace

import matplotlib.pyplot as plt
import cv2
import os, shutil, tqdm
import pandas as pd

def rgb_to_bgr(image):
    b, g, r = cv2.split(image)
    image = cv2.merge([r, g, b])
    return image

backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface', 'mediapipe']


In [ ]:
folder_dir = '/opt/ml/input/data_crop_test/train/images'
folder_name = os.listdir(folder_dir)
folder_name = sorted([x for x in folder_name if '._' not in x]) # '._' 제거

file_list = [[folder_name[i]+"/"+file for file in os.listdir(os.path.join(folder_dir,folder_name[i])) if '._' not in file and 'ipynb' not in file] for i in range(len(folder_name))]

# 모든 이미지명 리스트
all_file_list = list(chain(*file_list))

# class별 이미지명 리스트
incorrect_img = [os.path.join(folder_dir, file) for file in all_file_list if 'incorrect_mask' in file]
normal_img = [os.path.join(folder_dir, file) for file in all_file_list if 'normal' in file]
mask_img = [os.path.join(folder_dir, file) for file in all_file_list if 'incorrect_mask' not in file and 'normal' not in file]

print('모든 이미지 수: ', len(all_file_list))
print('incorrect 이미지 수: ', len(incorrect_img))
print('normal 이미지 수: ', len(normal_img))
print('mask 이미지 수: ', len(mask_img))
print('각 list 예시: ',incorrect_img[0], normal_img[0], mask_img[0], sep='\n')

normal_img = [file for file in normal_img if 'crop' not in file.split('/')[-1]]
print('normal 이미지 수: ', len(normal_img))

In [ ]:
fig, axes = plt.subplots(7, 5, figsize=(40, 24))
backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface', 'mediapipe']

for j in range(7):
    for i in range(5):
        face = DeepFace.detectFace(img_path = normal_img[j], target_size = (224, 224), detector_backend = backends[i])
        axes[j][i].imshow(face)
        axes[j][i].set_title(backends[i])
        axes[j][i].axis('off')


### SSD를 사용한 detection 
age, gender, boxxes

In [ ]:
detector = 'ssd'
data_normal_df = pd.DataFrame({}, columns=['file', 'age', 'region', 'gender', 'detector'])
fail = []

for image_file in tqdm.tqdm(normal_img, desc='iteration'):
    img_path, img_name = '/'.join(image_file.split('/')[:-1]), image_file.split('/')[-1]
    src, dst = img_name.split('.')
    
    try:
        # analyze
        obj = DeepFace.analyze(img_path = image_file, actions = ['age', 'gender'], detector_backend=detector)
        obj['file'] = image_file
        obj['detector'] = detector
        
        data_normal_df = data_normal_df.append(obj, ignore_index=True)
        
    except:
        fail.append(image_file)
    

In [ ]:
data_normal_df.head()
# data_normal_df.to_csv('detection_normal_ssd.csv', index=False)

### retinaface를 사용한 detection
age, gender, boxxes

In [ ]:
detector = backends[4] # retinaface
data_normal_df_retinaface = pd.DataFrame({}, columns=['file', 'age', 'region', 'gender', 'detector'])
fail = []

for image_file in tqdm.tqdm(normal_img, desc='iteration'):
    img_path, img_name = '/'.join(image_file.split('/')[:-1]), image_file.split('/')[-1]
    src, dst = img_name.split('.')
    
    try:
        # analyze
        obj = DeepFace.analyze(img_path = image_file, actions = ['age', 'gender'], detector_backend=detector)
        obj['file'] = image_file
        obj['detector'] = detector
        
        data_normal_df_retinaface = data_normal_df_retinaface.append(obj, ignore_index=True)
        
        # crop & save
        # face = DeepFace.detectFace(img_path = image_file, target_size = (112, 112), detector_backend = detector)
        # cv2.imwrite(os.path.join(img_path, src+"_crop_{}.".format(detector)+dst), detected_face)
    except:
        fail.append(image_file)
    

In [ ]:
data_normal_df_retinaface.head()
data_normal_df_retinaface.to_csv('detection_normal_retinaface.csv', index=False)